<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 8.5 - Prompting Large Language Models

In this lab we will practise prompting with a few Large Language Models (LLMs) using Groq (not to be confused with Grok). Groq is a platform that provides access to their custom-built AI hardware via APIs, allowing users to run open-source models such as Llama.

We shall see that while LLMs are powerful tools, how you ask a question or frame a task can dramatically influence the results obtained.

## Set-up

Step 1: Sign up for a free Groq account at https://console.groq.com/home .

Step 2: Create a new API key at https://console.groq.com/keys. Copy-paste it into an empty text file called 'groq_key.txt'.

Running the next cell will then read in this key and assign it to the variable `groq_key`.

In [3]:
groqfilename = r'groq_key.txt' # this file contains a single line containing your Groq API key only
try:
    with open(groqfilename, 'r') as f:
        groq_key = f.read().strip()
except FileNotFoundError:
    print("'%s' file not found" % filename)

In [7]:
!pip install groq

In [5]:
from groq import Groq
import requests
import pandas as pd
from IPython.display import Markdown

First create an instance of the Groq client:

In [7]:
client = Groq(api_key=groq_key)
client

The following code shows what models are currently accessible through Groq. `context_window` refers to the size of memory (in tokens) during a session and `max_completion_tokens` is the maximum number of tokens that are generated in an output.

In [9]:
url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {groq_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

pd.DataFrame(response.json()['data']).sort_values(['created'], ascending=False)

,id,object,created,owned_by,active,context_window,public_apps,max_completion_tokens
16,meta-llama/llama-prompt-guard-2-86m,model,1748632165,Meta,True,512,None,512
12,meta-llama/llama-prompt-guard-2-22m,model,1748632101,Meta,True,512,None,512
5,qwen/qwen3-32b,model,1748396646,Alibaba Cloud,True,131072,None,40960
0,meta-llama/llama-guard-4-12b,model,1746743847,Meta,True,131072,None,1024
19,meta-llama/llama-4-maverick-17b-128e-instruct,model,1743877158,Meta,True,131072,None,8192
6,meta-llama/llama-4-scout-17b-16e-instruct,model,1743874824,Meta,True,131072,None,8192
3,compound-beta-mini,model,1742953279,Groq,True,131072,None,8192
14,qwen-qwq-32b,model,1741214760,Alibaba Cloud,True,131072,None,131072
17,compound-beta,model,1740880017,Groq,True,131072,None,8192
10,playai-tts-arabic,model,1740682783,PlayAI,True,8192,None,8192


The Groq client object enables interaction with the Groq REST API and a chat completion request is made via the client.chat.completions.create method.

The most important arguments of the client.chat.completions.create method are the following:
* messages: a list of messages (dictionary form) that make up the conversation to date
* model: a string indicating which model to use (see [list of models](https://console.groq.com/docs/models))
* max_completion_tokens: the maximum number of tokens that are generated in the chat completion
* response_format: setting this to `{ "type": "json_object" }` enables JSON output
* seed: sample deterministically as best as possible, though identical outputs each time are not guaranteed
* temperature: between 0 and 2 where higher values like 0.8 make the output more random (creative) and values like 0.2 are more focused and deterministic


In [17]:
help(client.chat.completions.create)

Help on method create in module groq.resources.chat.completions:

create(*, messages: 'Iterable[ChatCompletionMessageParam]', model: "Union[str, Literal['gemma2-9b-it', 'llama-3.3-70b-versatile', 'llama-3.1-8b-instant', 'llama-guard-3-8b', 'llama3-70b-8192', 'llama3-8b-8192']]", exclude_domains: 'Optional[List[str]] | NotGiven' = NOT_GIVEN, frequency_penalty: 'Optional[float] | NotGiven' = NOT_GIVEN, function_call: 'Optional[completion_create_params.FunctionCall] | NotGiven' = NOT_GIVEN, functions: 'Optional[Iterable[completion_create_params.Function]] | NotGiven' = NOT_GIVEN, include_domains: 'Optional[List[str]] | NotGiven' = NOT_GIVEN, logit_bias: 'Optional[Dict[str, int]] | NotGiven' = NOT_GIVEN, logprobs: 'Optional[bool] | NotGiven' = NOT_GIVEN, max_completion_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN, max_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN, metadata: 'Optional[Dict[str, str]] | NotGiven' = NOT_GIVEN, n: 'Optional[int] | NotGiven' = NOT_GIVEN, parallel_tool_calls:

As a first example, note how the messages input is given as a list of a dictionaries with `role` and `content` keys. This is in a ChatML format recognised by many LLMs.

In [21]:
chat_completion = client.chat.completions.create(
    messages=[
        {   "role": "system", # sets the persona of the model
            "content": "You are a helpful assistant."
        },
        {
            "role": "user", # what the user wants the assistant to do
            "content": "Explain briefly how large language models work",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Large language models are artificial intelligence (AI) systems that process and generate human-like language. They work by:

1. **Training on vast amounts of text data**: The model is trained on a massive dataset of text, which allows it to learn patterns, relationships, and structures of language.
2. **Learning to predict the next word**: The model is trained to predict the next word in a sequence, given the context of the previous words. This is done using complex algorithms and neural networks.
3. **Generating text based on context**: Once trained, the model can generate text by predicting the next word, given a prompt or context. This process is repeated to create coherent and natural-sounding text.

The key technologies behind large language models include:

* **Neural networks**: Complex algorithms that mimic the human brain's ability to learn and process information.
* **Deep learning**: A subset of machine learning that uses neural networks to analyze and generate data.
* **Tra

The output is in Markdown format so the following line formats this text.

In [23]:
Markdown(chat_completion.choices[0].message.content)

Large language models are artificial intelligence (AI) systems that process and generate human-like language. They work by:

1. **Training on vast amounts of text data**: The model is trained on a massive dataset of text, which allows it to learn patterns, relationships, and structures of language.
2. **Learning to predict the next word**: The model is trained to predict the next word in a sequence, given the context of the previous words. This is done using complex algorithms and neural networks.
3. **Generating text based on context**: Once trained, the model can generate text by predicting the next word, given a prompt or context. This process is repeated to create coherent and natural-sounding text.

The key technologies behind large language models include:

* **Neural networks**: Complex algorithms that mimic the human brain's ability to learn and process information.
* **Deep learning**: A subset of machine learning that uses neural networks to analyze and generate data.
* **Transformer architecture**: A specific type of neural network architecture that is well-suited for natural language processing tasks.

These models can perform a wide range of tasks, including language translation, text summarization, and conversation generation.

## Text summarisation

We start with a llama3-8b-8192, a model using just over 8 billion parameters with at most 8192 tokens produced as output.

Here is an article to be summarised from the [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset:

In [10]:
story = """
SAN FRANCISCO, California (CNN) -- A magnitude 4.2 earthquake shook the San Francisco area Friday at 4:42 a.m. PT (7:42 a.m. ET), the U.S. Geological Survey reported. The quake left about 2,000 customers without power, said David Eisenhower, a spokesman for Pacific Gas and Light. Under the USGS classification, a magnitude 4.2 earthquake is considered "light," which it says usually causes minimal damage. "We had quite a spike in calls, mostly calls of inquiry, none of any injury, none of any damage that was reported," said Capt. Al Casciato of the San Francisco police. "It was fairly mild." Watch police describe concerned calls immediately after the quake » . The quake was centered about two miles east-northeast of Oakland, at a depth of 3.6 miles, the USGS said. Oakland is just east of San Francisco, across San Francisco Bay. An Oakland police dispatcher told CNN the quake set off alarms at people's homes. The shaking lasted about 50 seconds, said CNN meteorologist Chad Myers. According to the USGS, magnitude 4.2 quakes are felt indoors and may break dishes and windows and overturn unstable objects. Pendulum clocks may stop.
"""

**Exercise:**
Summarise the story text using the following three prompts. Use the format given above but here there is no need to set the persona (i.e. only include one dictionary in the messages list when calling `client.chat.completions.create`.) Comment on any differences.

1) "Summarise the following article in 3 sentences."

2) "Give me a TL;DR of this text."

3) "What's the key takeaway here?"

In [12]:
prompts = ["Summarise the following article in exactly 3 sentences. ", "Give me a TL;DR of this text. ", "What's the key takeaway here?"] #update the prompt to 'exactly 3 sentences' as it was outputting 4 sentences.
#content will be p + story for p in prompts
for p in prompts:
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user", # what the user wants the assistant to do
            "content": p + story
        }
    ],
    model="llama-3.3-70b-versatile",
)

    
print(f"Prompt: {p}")
print("Response:", response.choices[0].message.content)


Prompt: What's the key takeaway here?
Response: A magnitude 4.2 earthquake struck the San Francisco area, causing about 2,000 power outages, but no injuries or significant damage were reported, with the quake being described as "light" and "fairly mild".


Run the above code again below and note that the answers may differ. This is due to the probabilistic nature of LLM token generation.

In [14]:
# ANSWER
for p in prompts:
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user", # what the user wants the assistant to do
            "content": p + story
        }
    ],
    model="llama-3.3-70b-versatile",
)

    
print(f"Prompt: {p}")
print("Response:", response.choices[0].message.content)

Prompt: What's the key takeaway here?
Response: A 4.2 magnitude earthquake struck the San Francisco area, causing minimal damage and no reported injuries, with approximately 2,000 customers losing power.


## Text completion

**Exercise**: In this section adjust the `max_completion_tokens` and `temperature` settings below to obtain different responses. Show some examples with the prompt "Continue the story: It was a great time to be alive" with the model "llama-3.1-8b-instant".

* max_completion_tokens - the maximum number of tokens to generate. Note that longer words are made of multiple tokens (set to 200 and 500)
* temperature (positive number) - the higher the number the more random (creative) the output (set to 0.2, 0.8, 2)

In [16]:
# ANSWER (set max_completion_tokens=200, do not have a temperature setting)
for p in prompts:
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user", # what the user wants the assistant to do
            "content": p + story
        }
    ],
    model="llama-3.1-8b-instant",
    max_completion_tokens=200
)

    
print(f"Prompt: {p}")
print("Response:", response.choices[0].message.content)

Prompt: What's the key takeaway here?
Response: The key takeaway is that a magnitude 4.2 earthquake occurred in the San Francisco area, causing disruptions such as power outages and numerous calls to authorities, but no reported injuries or significant damage due to the relatively mild intensity of the quake.


In [18]:
# ANSWER (set max_completion_tokens=500, do not have a temperature setting)
for p in prompts:
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user", # what the user wants the assistant to do
            "content": p + story
        }
    ],
    model="llama-3.1-8b-instant",
    max_completion_tokens=500
)

    
print(f"Prompt: {p}")
print("Response:", response.choices[0].message.content)

Prompt: What's the key takeaway here?
Response: The key takeaway is that a magnitude 4.2 earthquake occurred in the San Francisco area at 4:42 a.m., resulting in minimal damage and no reported injuries. It caused a power outage for 2,000 customers and triggered concerns but overall was described as a minor, "light" quake that only lasted 50 seconds.


In [20]:
# ANSWER (set temperature = 0.2, do not have a max_completion_tokens setting)
for p in prompts:
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user", # what the user wants the assistant to do
            "content": p + story
        }
    ],
    model="llama-3.1-8b-instant",
    temperature=0.2
)

    
print(f"Prompt: {p}")
print("Response:", response.choices[0].message.content)

Prompt: What's the key takeaway here?
Response: The key takeaway is that a magnitude 4.2 earthquake occurred in the San Francisco area, causing minimal damage and no reported injuries, but leaving about 2,000 customers without power and triggering alarm systems at homes.


In [24]:
# ANSWER (set temperature = 1, do not have a max_completion_tokens setting)
for p in prompts:
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user", # what the user wants the assistant to do
            "content": p + story
        }
    ],
    model="llama-3.1-8b-instant",
    temperature=1
)

    
print(f"Prompt: {p}")
print("Response:", response.choices[0].message.content)

Prompt: What's the key takeaway here?
Response: The key takeaway is that a magnitude 4.2 earthquake occurred in the San Francisco area, causing minimal damage and no reported injuries. The quake, which lasted about 50 seconds, left around 2,000 customers without power, but overall it's considered a "light" earthquake with typical effects such as breaking dishes and overturning unstable objects.


Note what happens when the temperature is set too high!

In [26]:
# ANSWER (set temperature = 2, do not have a max_completion_tokens setting)
for p in prompts:
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user", # what the user wants the assistant to do
            "content": p + story
        }
    ],
    model="llama-3.1-8b-instant",
    temperature=2
)

    
print(f"Prompt: {p}")
print("Response:", response.choices[0].message.content)

Prompt: What's the key takeaway here?
Response: A moderate earthquake (4.2 magnitude) in the San Francisco area caused mostly minor concerns, but minimal reported damage or injuries despite affecting around 2,000 PG&E power customers.


### Zero-shot and one-short prompting for question-answering

This section shows the impact of prompting on the response. Zero-shot prompting means we provide the prompt without any examples or additional context. Let us initially ask Mistral a question using no prompting.

In [30]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "How do two chemicals react?"}],
    temperature = 0.8,
)

Markdown(response.choices[0].message.content)

The reaction between two chemicals can be complex and depends on various factors, such as the type of chemicals involved, the conditions under which they are brought together, and the energy available. Here's a general overview of how two chemicals react:

1. **Collision Theory**: For a chemical reaction to occur, the molecules of the two substances must collide with each other. The collision must be energetic enough to break the bonds between the atoms in the molecules and form new bonds.
2. **Activation Energy**: The energy required for the molecules to collide and react is called the activation energy. If the energy is too low, the molecules will simply bounce off each other, and no reaction will occur.
3. **Chemical Reaction Types**: There are several types of chemical reactions, including:
	* **Synthesis reaction**: Two molecules combine to form a new molecule (e.g., H2 + O2 → H2O).
	* **Decomposition reaction**: A single molecule breaks down into two or more molecules (e.g., 2H2O → 2H2 + O2).
	* **Replacement reaction**: One molecule replaces another molecule in a compound (e.g., Zn + CuSO4 → ZnSO4 + Cu).
	* **Combustion reaction**: A molecule reacts with oxygen to produce heat and light (e.g., CH4 + 2O2 → CO2 + 2H2O).
4. **Reaction Mechanism**: The reaction mechanism is the sequence of steps that occurs during the reaction. It can be a single step or multiple steps, and it may involve the formation of intermediate molecules.
5. **Factors Affecting Reaction Rate**: The rate of a chemical reaction can be influenced by various factors, including:
	* **Concentration**: Increasing the concentration of one or both reactants can increase the reaction rate.
	* **Temperature**: Increasing the temperature can increase the reaction rate, but it can also lead to unwanted side reactions.
	* **Catalysts**: Adding a catalyst can increase the reaction rate by lowering the activation energy.
	* **Surface area**: Increasing the surface area of one or both reactants can increase the reaction rate.

To understand how two chemicals react, you need to know the chemical formulas and properties of the substances, as well as the conditions under which they are brought together. You can use various tools, such as:

* **Chemical equation**: A chemical equation is a shorthand way of representing a chemical reaction.
* **Balanced equation**: A balanced equation is a chemical equation in which the number of atoms of each element is the same on both the reactant and product sides.
* **Reaction diagram**: A reaction diagram is a graphical representation of a chemical reaction, showing the reactants, products, and intermediate molecules.

Here's an example of a simple chemical reaction:

**Reaction:** H2 + O2 → H2O

**Reactants:** Hydrogen gas (H2) and oxygen gas (O2)

**Products:** Water (H2O)

**Conditions:** The reaction occurs when the hydrogen and oxygen gases are brought together in the presence of a spark or heat.

**Mechanism:** The reaction involves the collision of a hydrogen molecule with an oxygen molecule, resulting in the formation of a water molecule.

**Factors affecting reaction rate:** The reaction rate can be affected by the concentration of the reactants, the temperature, and the presence of a catalyst.

Keep in mind that this is a simplified example, and the actual reaction mechanism may be more complex.

**Exercise:** Ask the same question but modify the prompt to return the answer to the same question in a simpler form (still using the llama-3.1-8b-instant model). Experiment with different prompts.

In [35]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "How do chemicals react? Explain this using easy to understand lament terms."}],
    temperature = 0.8,
)

Markdown(response.choices[0].message.content)


Chemical reactions are like cooking recipes in a lab. Imagine you have different ingredients, like food or building blocks, that combine in specific ways to create something new.

**Chemical Reaction Basics:**

1. **Reactants**: These are the ingredients or substances that start the reaction.
2. **Products**: These are the new substances created by combining the reactants.
3. **Chemical Change**: This is when the reactants transform into products, changing their properties or appearance.

**Types of Chemical Reactions:**

1. **Combination Reaction**: This is like mixing ingredients to create something new. For example, making a cake by combining flour, sugar, eggs, and milk.
2. **Decomposition Reaction**: This is like breaking down ingredients to create something simpler. For example, when you leave food out and it spoils, breaking down into simpler compounds.
3. **Single Element Reaction**: This is like combining two elements, like hydrogen and oxygen (H2 + O2) to create water (H2O).
4. **Displacement Reaction**: This is like replacing one element with another. For example, when iron (Fe) is added to copper oxide, it replaces the copper (Cu) to form iron oxide (Fe2O3).

**Factors That Influence Reactions:**

1. **Temperature**: Heat can speed up or slow down reactions. Think of a cake recipe that requires baking at 350°F.
2. **Pressure**: Applying pressure can affect the rate of a reaction. Imagine squeezing a bottle of soda to create more pressure.
3. **Light**: Some reactions require light energy, like photosynthesis in plants.
4. **Catalysts**: These are substances that help speed up reactions without being used up. Think of a chef's favorite spice that enhances flavor.

**Remember:**

* Chemical reactions involve changes in the arrangement of particles, creating new substances.
* Factors like temperature, pressure, and light can influence reactions.
* Understanding chemical reactions can help us create new products, like medicines, fuels, and food.

Now, you have a basic understanding of chemical reactions. It's like cooking, but with atoms and molecules instead of ingredients!

### One-shot prompting ###

Next, note the dramatic change when we give the following template setting a new role and providing an English question followed by a French translation.

In [64]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "system",
             "content": "You translate English to French."},
              {"role": "user",
               "content": "What time is it?"},
               {"role": "assistant",
               "content": "Quelle heure est-il?"},
              {"role": "user",
               "content": "How do two chemicals react?"}],
    temperature = 0.8,
)
print(response.choices[0].message.content)

Comment deux composés chimiques réagissent-ils ?


### Few-shot prompting

Recall that since the text generation process outputs one token at a time, their outputs often need adjusting. This is where examples can help.

In [60]:
prompt1 = "I'm gonna head out now, see you later."
response1 = "I will be leaving now. See you later."

prompt2 =  "That movie was super cool!"
response2 = "The movie was very impressive."

prompt3 = "Can't make it to the meeting, sorry."


response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a professional editor. Rewrite casual sentences into a formal tone."},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": response2},
        {"role": "user", "content": prompt3},
    ]
)

print(response.choices[0].message.content.strip())


I regret to inform you that I will be unable to attend the meeting.


The output can also be moulded to provide SQL output.

In [66]:
prompt1 = "Show me all users who signed up in the last 30 days."
response1 = "SELECT * FROM users WHERE signup_date >= CURRENT_DATE - INTERVAL '30 days';"

prompt2 = "What is the average order value?"
response2 =  "SELECT AVG(order_total) FROM orders;"

prompt3 = "List products that are out of stock."

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are an assistant that translates natural language to SQL."},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": response2},
        {"role": "user", "content": prompt3},
    ]
)

print(response.choices[0].message.content.strip())


SELECT * FROM products WHERE quantity_in_stock = 0;


**Exercise**: Create a few examples to train the "llama3-70b-8192" LLM to take in user content in the form below and provide output as a pandas dataframe. Use the `exec` function to execute its output to display the answer of sample input as a data frame.

Example:

given the user content

"""

| col1 | col2 | col3

| 32 | 27 | 25

| 64 | 23 | 14

"""

train the model to output

df = pd.DataFrame({'col1': [32, 64], 'col2': [27, 23], 'col3': [25, 14]})



In [87]:
example1 = '''
| col1 | col2 | col3
| 32   | 27   | 25
| 64   | 23   | 14
'''

output1 = '''
df = pd.DataFrame({'col1': [32, 64], 'col2': [27, 23], 'col3': [25, 14]})
'''
example2 = '''
| A | B    | C
| 4 | 2333 | 78
| 2 | 222  | 65
'''

output2 = '''
df = pd.DataFrame({'A': [4, 2], 'B': [2333, 222], 'C': [78, 65]})
'''

example3 = """
| Name   | Age | Score
| Alice  | 25  | 88
| Bob    | 30  | 92
| Carol  | 22  | 95
"""

output3 = '''
df = pd.DataFrame({'Name': ['Alice', 'Bob', 'Carol'], 'Age': [25, 30, 22], 'Score': [88, 92, 95]})
'''
response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "Use the examples to create a dataframe identified as df and to execute the output"},
        {"role": "user", "content": example1},
        {"role": "assistant", "content": output1},
        {"role": "user", "content": example2},
        {"role": "assistant", "content": output2},
        {"role": "user", "content": example3},
    ]
)

exec(response.choices[0].message.content.strip()) # string executed as Python code
df

,Name,Age,Score
0,Alice,25,88
1,Bob,30,92
2,Carol,22,95


Also show what happens when the question is asked in the absence of a system role and without few-shot prompting.

In [91]:
response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
       {"role": "user", "content": example3},
    ]
)

response.choices[0].message.content.strip() # string executed as Python code

"A nice little table!\n\nIt looks like we have a table with three columns: `Name`, `Age`, and `Score`. And three rows of data, each representing a person.\n\nHere's a quick summary of the data:\n\n* The names are Alice, Bob, and Carol.\n* Their ages are 25, 30, and 22, respectively.\n* Their scores are 88, 92, and 95, respectively.\n\nIs there anything specific you'd like to do with this data or ask about it?"

### Chain-of-thought prompting

The results of question-answering can also be improved by prompting the LLM to provide intermediate steps.

**Exercise**: Using the following prompts, compare the answers of the "llama3-8b-8192" model (set seed=21). (If this model is no longer available choose a model with relatively few parameters.)

zero_shot_prompt = "How many s's are in the word 'success'?"

chain_of_thought_prompt = "How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn."

In [113]:
zero_shot_prompt = "How many s's are in the word 'success'?"
chain_of_thought_prompt = "How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn."

response1 = client.chat.completions.create(
    model="llama3-8b-8192",
    seed=21,
    messages=[
       {"role": "user", "content": zero_shot_prompt},
    ]
)

response2 = client.chat.completions.create(
    model="llama3-8b-8192",
    seed=21,
    messages=[
       {"role": "user", "content": chain_of_thought_prompt},
    ]
)

print('Zero shot:\n',response1.choices[0].message.content,'\n')
print('Chain of thought:\n', response2.choices[0].message.content)

Zero shot:
 There are 2 s's in the word "success". 

Chain of thought:
 Let's go through each letter of the word "success" to count the number of s's:

1. S (first letter)
2. U
3. C
4. C
5. E
6. S
7. S

As we go through the letters, we find two S's (at positions 1 and 6) and three other letters. Therefore, the correct answer is:

There are 2 S's in the word "success".


## Comparison of LLMs

**Exercise**: Compare the performance of 2 LLMs by outputting the answers of the following questions into a dataframe.

    "Tell me a joke about data science.",
    "How can one calculate 22 * 13 mentally?",
    "Write a creative story about a baby learning to crawl.",

Column headings:

Model Name | Question | Answer

In [16]:
pd.set_option('display.max_colwidth', None) # allows wide dataframes to be viewed
models = models = {
    "gemma2-9b-it": "Gemma2 9b",
    "llama3-8b-8192": "LLaMA 3 8B"
} #can edit this

# ANSWER
prompts = ["Tell me a joke about data science.",
"How can one calculate 22 * 13 mentally?",
"Write a creative story about a baby learning to crawl.",]

results = []

for model_id, model_name in models.items():
    for prompt in prompts:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            )
        answer = response.choices[0].message.content.strip()
        results.append({
            "Model Name": model_name,
            "Question": prompt,
            "Answer": answer
        })

In [18]:
df = pd.DataFrame(results)
df

,Model Name,Question,Answer
0,Gemma2 9b,Tell me a joke about data science.,Why did the data scientist break up with the statistician? \n\nBecause they had too many standard deviations! 😂 \n\n\nLet me know if you'd like to hear another one! 😄
1,Gemma2 9b,How can one calculate 22 * 13 mentally?,"Here's a way to calculate 22 * 13 mentally:\n\n**Break it down:**\n\n* **Think of 22 as (20 + 2)**\n* **Use the distributive property:** \n * 22 * 13 = (20 + 2) * 13 \n * = 20 * 13 + 2 * 13\n\n* **Calculate the simpler parts:**\n * 20 * 13 = 260\n * 2 * 13 = 26\n\n* **Add the results:**\n * 260 + 26 = 286\n\n\n**Therefore, 22 * 13 = 286**"
2,Gemma2 9b,Write a creative story about a baby learning to crawl.,"Bartholomew ""Barry"" Butterfield III, a chubby, cherub-faced baby with a shock of unruly ginger hair, stared intently at the magnificent, towering structure that was his mother's favorite houseplant. It was a fern, a leafy green behemoth that reached almost to the coffee table, its fronds unfurling like emerald flags in the afternoon sun. \n\nBarry coveted the fern. Not for its exotic greenery, but for its precarious position atop a wobbly toy box. Every morning, his mother would perch the box on the rug, tempting him with the forbidden land of soft-plush toys and crinkly treat bags. And just beyond that tempting space, the fern. It felt like a magical kingdom just out of reach. \n\nFor weeks, Barry had practiced pushing and scooting himself with his chubby arms. He'd propel himself backward, forwards, and sometimes even sideways, but never onward to the glorious fern. He'd watch his older sister, Penelope, glide across the room with the ease of a newborn gazelle, and scoff. Crawling was for show! He would conquer it on his own terms, in a feat of unparalleled baby brilliance. \n\nOne sunny morning, a determined glint ignited in Barry's baby blue eyes. He positioned himself, feet planted firmly, then, with a grunt and a wiggle, thrust his arms forward. His face scrunched in concentration, his bottom lifting and lowering like a mini marshmallow on a hot pan. \n\nHe moved! It was a jerky, stuttering dance, his arms flailing wildly, his legs splaying like a confused spider. But he was moving! He was creeping towards his coveted target!\n\nHis mother, who had been diligently working on a cross-stitch project, watched in amusement. \n\n“That’s it, Barry! You’re doing it!” \n\nHer cheery encouragement only fueled Barry’s determination. He forgot his wobbly gait, his awkward limbs, and his mother's increasingly bewildered narration of his journey. He was on a mission! His tiny brow furrowed, his tongue poked out in concentration.\n\nFinally, after what seemed like an eternity (to Barry, at least), he reached the toy box. His chest heaved, his tiny heart pounded with triumph. He had conquered the first obstacle!\n\nBut the fern still loomed, a verdant promise just beyond. With renewed vigor, Barry pulled himself toward the leafy giant, his legs churning, his arms reaching, his face a mask of sweaty, triumphant determination.\n\nAt last, he reached the fern’s base. He leaned forward, his chubby hand reaching out to touch the delicate fronds. He had done it! He had climbed Mount Fern! \n\nBarry’s mother, tears welling up in her eyes, scooped him into her arms.\n\n""My brave little explorer!"" she exclaimed.\n\nBarry, exhausted but proud, gurgled happily, nibbling at his mother’s cheek. The world felt full of adventures, and there were so many more ferns to conquer."
3,LLaMA 3 8B,Tell me a joke about data science.,Why did the data scientist quit his job?\n\n porque he didn't get the correlation he was looking for!
4,LLaMA 3 8B,How can one calculate 22 * 13 mentally?,"There are several ways to calculate 22 * 13 mentally, and here are a few:\n\n**Method 1: Breaking down the calculation**\nBreak down the calculation into two parts: 20 * 13 and 2 * 13. Then add the results:\n20 * 13 = 260 (quickly multiply 20 by 10 = 200, then add 3 x 20 = 60)\n2 * 

### Bonus

See if you can prompt an LLM to perform sentiment analysis (output 'Positive' or 'Negative' only) on a given piece of text.

In [25]:
response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
       {"role": "user", "content": "Is this text Positive or Negative? Answer only with 'Positive' or 'Negative'.\nText: I love this product!\nSentiment:"},
    ]
)

response.choices[0].message.content.strip()

'Positive'

In [27]:

def get_sentiment(text: str, model_id="llama3-8b-8192") -> str:
    prompt = (
        f"Classify the sentiment of the following text as either 'Positive' or 'Negative' only.\n\n"
        f"Text: \"{text}\"\n\n"
        f"Sentiment:"
    )

    response = client.chat.completions.create(
        model=model_id,
        messages=[{"role": "user", "content": prompt}],
        )

    sentiment = response.choices[0].message.content.strip()
    # Optional: ensure output is exactly 'Positive' or 'Negative'
    if sentiment.lower() in ['positive', 'negative']:
        return sentiment.capitalize()
    else:
        # fallback or retry could go here
        return "Unknown"

# Example usage
text_samples = [
    "I love how easy this product is to use!",
    "The service was terrible and the staff were rude.",
    "This coding is real hard.",
]

for text in text_samples:
    sentiment = get_sentiment(text)
    print(f"Text: {text}\nSentiment: {sentiment}\n")

Text: I love how easy this product is to use!
Sentiment: Positive

Text: The service was terrible and the staff were rude.
Sentiment: Unknown

Text: This coding is real hard.
Sentiment: Negative



## Conclusion

We worked with a few Large Language Models (LLMs) using Groq and experimented with prompting for summarisation, text completion and question-answering tasks.

We also explored controlling the randomness (creativity) of output through the temperature setting and tried different types of prompting to achieve desired forms of output.

## References
1. [Groq's prompting guide](https://console.groq.com/docs/prompting)
2. [Groq's playground](https://console.groq.com/playground)



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



